In [ ]:
sample_name = "SRR13329143"

In [ ]:
# Parameters
sample_name = "SRR13329175"


In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

In [ ]:
import gc
import scanpy as sc
import muon as mu
import seaborn as sns
import os
import pandas as pd
from matplotlib import pyplot as plt

os.environ['R_HOME'] = '/gpfs/bwfor/work/ws/hd_fu399-conda/conda/envs/python_R/lib/R/'
import anndata2ri
import logging

import rpy2.rinterface_lib.callbacks as rcb
import rpy2.robjects as ro

rcb.logger.setLevel(logging.ERROR)
ro.pandas2ri.activate()
anndata2ri.activate()

%load_ext rpy2.ipython

In [ ]:
%%R

suppressMessages(library(SeuratDisk))
suppressMessages(library(SingleR))
suppressMessages(library(Seurat))
suppressMessages(library(SingleCellExperiment))
suppressMessages(library(UCell))

In [ ]:
dataset = sc.read(f"../../int_data/{sample_name}_qc.h5ad")
dataset.X = dataset.layers["soupX_counts"]

In [ ]:
dataset.obs[["AvgSpotLen", "Bases", "Bytes", "version"]] = dataset.obs[["AvgSpotLen", "Bases", "Bytes", "version"]].astype("str")

In [ ]:
%%R -i dataset

sce <- SingleCellExperiment(list(counts=dataset@assays@data[["soupX_counts"]]),
    colData=dataset@colData,
)

row.names(sce) <- row.names(dataset)

data <- as.Seurat(sce, data = NULL)

In [ ]:
%%R
data

In [ ]:
%%R

data <- NormalizeData(data)
data <- ScaleData(data, features = row.names(data))
data <- FindVariableFeatures(data)
data <- RunPCA(data, verbose = F)
data <- FindNeighbors(data, verbose = F)
data <- FindClusters(data, verbose = F)
data <- RunUMAP(data, dims = 1:20, verbose = F)

In [ ]:
%%R

ref1 <- celldex::BlueprintEncodeData()
ref2 <- celldex::MonacoImmuneData()
cell_classification <- SingleR(test = as.SingleCellExperiment(data),
                               ref = list(BPE = ref1, MON = ref2),
                               labels = list(ref1$label.main, ref2$label.main))

data[["SingleR_label"]] <- cell_classification$labels

plotScoreHeatmap(cell_classification)

In [ ]:
%%R -i sample_name

write.csv(data@meta.data, paste0(sample_name, "_metadata.csv"))